# Preparing Music Data

In [113]:
from music21 import converter, instrument, note, chord
import glob

In [114]:
notes = [] #Data sequence

for file in glob.glob("BattleMusic/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    
    parts = instrument.partitionByInstrument(midi)
    
    if parts: #file has more than one instrument
        notes_to_parse = parts.parts[0].recurse()
    else: # notes are flat
        notes_to_parse = midi.flat.notes
    
    for element in notes_to_parse:
        if isinstance(element, note.Note): #if it's a note
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            


In [115]:
print(len(notes))

53208


# Map Notes/Chords to Integers

In [116]:
sequence_length = 100

#get all pitch names
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

#create input sequences and their corresponding output sequences

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

#nb_classes = len(note_to_int) #number of classifications
n_patterns = len(network_input) #number of example sequences

# Prepping for Network

In [117]:
import numpy as np
import tensorflow as tf
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint

In [118]:
#reshape input into shapes the LSTM can take
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input to values 0<= x <= 1
network_input = network_input / float(nb_classes)

#One hot encoding the output

network_output = np_utils.to_categorical(network_output) 
print(network_output.shape)

(53108, 282)


# Building the Model

In [132]:
model = Sequential() #We are creating a sequential model
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(282))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_76 (LSTM)               (None, 100, 256)          264192    
_________________________________________________________________
dropout_76 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
lstm_77 (LSTM)               (None, 100, 512)          1574912   
_________________________________________________________________
dropout_77 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_78 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_49 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_78 (Dropout)         (None, 256)               0         
__________

In [133]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    

callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, 
          batch_size=64, callbacks=callbacks_list)

Epoch 1/200
 6208/53108 [==>...........................] - ETA: 32:20 - loss: 4.5858

KeyboardInterrupt: 

In [122]:
model = Sequential() #We are creating a sequential model
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(282))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node

model.load_weights('weights-improvement-15-1.8852-bigger.hdf5')

In [127]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(nb_classes)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

In [128]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [131]:
from music21 import stream

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'